In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import lightgbm as lgb
import seaborn as sns
import json
#画像処理ライブラリ　※今回はテーブルデータのみを使用
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
get_ipython().run_line_magic("matplotlib","inline")

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms

In [ ]:
train_df = pd.read_csv("../input/petfinder-adoption-prediction/train/train.csv")
test_df = pd.read_csv("../input/petfinder-adoption-prediction/test/test.csv")
sub = pd.read_csv("../input/petfinder-adoption-prediction/test/sample_submission.csv")

trainデータを確認



In [ ]:
print(train_df.shape)
train_df.head()

ほとんどが数値データとなっている。

objectはName,RescuerID,Description,PetIDとなっている。

In [ ]:
train_df.info()

欠損値の確認

NameとDiscriptionに欠損値が存在する。
今回、Nameは列を削除、Descriotionが欠損のものは件数が少ないため省いて計算する。

In [ ]:
train_df.isnull().sum()

次にtestデータの確認をする。

In [ ]:
print(test_df.shape)
test_df.head()

In [ ]:
test_df.info()

In [ ]:
test_df.isnull().sum()

adoptionspeedを確認

0が極端に少なく５００弱データ。

１、３は３０００データ、２、４は４０００データ。

In [ ]:
print(sns.countplot(train_df["AdoptionSpeed"]))

In [ ]:
train_df.describe()

Ageは大きく外れている値がある。外れ値として除外するのも検討する必要がある。

In [ ]:
print(sns.countplot(train_df["Age"]))

In [ ]:
print(sns.countplot(train_df["Quantity"]))

処理したものを入れていくためにfeaturesデータを作成

In [ ]:
train_features = train_df.copy()
test_features = test_df.copy()

まずはName,RescuerID,Description,PetIDを削除

In [ ]:
train_features = train_features.drop(["Name","RescuerID","Description","PetID"],axis=1)
test_featires = test_features.drop(["Name","RescuerID","Description","PetID"],axis=1)

StratifiedKFoldでlightgbmを実施

In [ ]:
X = train_features.drop("AdoptionSpeed",axis=1)
y = train_features["AdoptionSpeed"]

In [ ]:
FOLD = 5
NUM_ROUND = 100
VERBOSE_EVAL = -1
NUM_CLASS = 3

params = {
    'objective': 'multiclass',
    'verbose': -1,
    'num_class': NUM_CLASS
}

valid_scores = []
models = []
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(kf)

In [ ]:
print(X.shape)
print(y.shape)
X[-1:]

In [ ]:
from sklearn.metrics import log_loss

FOLD = 5
NUM_ROUND = 100
VERBOSE_EVAL = -1
NUM_CLASS = 5

params = {
    'objective': 'multiclass',
    'verbose': -1,
    'num_class': NUM_CLASS
}

valid_scores = []
models = []
kf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=42)
kf.get_n_splits(X,y)

for fold, (train_indices, valid_indices) in enumerate(kf.split(X, y)):
    print("TRAIN:",train_indices,"TEST:",valid_indices)
    X_train, X_valid = X.iloc[list(train_indices)], X.iloc[list(valid_indices)]
    y_train, y_valid = y.iloc[list(train_indices)], y.iloc[list(valid_indices)]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    score = log_loss(y_valid, y_valid_pred)
    print(f'fold {fold} log loss: {score}')
    valid_scores.append(score)

    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

CV mean スコア1.34

次は特徴量にオブジェクトを追加する。

DescriotionをCountVectorizerを用いてベクトル化

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# token_patternは、一つの文字のまとまりをどのように抽出するかを、正規表現で指定するためのオプション
# 正規表現を用いて'(?u)\\b\\w+\\b'という表現をすることにより、スペース単位で文字を区切ることができる
train_df2 = train_df.dropna(subset=['Description'])
print(train_df.shape)
bow_converter = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
bow_converter.fit(train_df2['Description'])

In [ ]:
words = bow_converter.get_feature_names() # 要素名を取得
words[:10]

In [ ]:
#test_dfのDescriptionのNan行をhinanに格納し確認
hinan = test_df[test_df["Description"].isnull()]
print(hinan)
print(hinan["Description"].unique())

In [ ]:
#test_dfのカラムでDescriptionでnanをaを代入
test_df2 = test_df["Description"].fillna("a")

In [ ]:
test_df2

In [ ]:
# 各レコードごとに、単語の出現回数を計算
train_words = bow_converter.transform(train_df2['Description'])
test_words = bow_converter.transform(test_df2)

In [ ]:
print(train_words.shape)
print(train_df2.shape)

In [ ]:
#train_wordsをtrain_features,test__featureに代入
train_features[words] = pd.DataFrame(train_words.toarray(), columns=words)
test_features[words] = pd.DataFrame(test_words.toarray(), columns=words)

In [ ]:
test_features2 = test_features.drop(["Name","RescuerID","Description","PetID"],axis=1)
test_features2.head()

In [ ]:
X = train_features.drop("AdoptionSpeed",axis=1)
y = train_features["AdoptionSpeed"]

In [ ]:
FOLD = 5
NUM_ROUND = 100
VERBOSE_EVAL = -1
NUM_CLASS = 5

params = {
    'objective': 'multiclass',
    'verbose': -1,
    'num_class': NUM_CLASS
}

valid_scores = []
models = []
kf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=42)
kf.get_n_splits(X,y)

for fold, (train_indices, valid_indices) in enumerate(kf.split(X, y)):
    print("TRAIN:",train_indices,"TEST:",valid_indices)
    X_train, X_valid = X.iloc[list(train_indices)], X.iloc[list(valid_indices)]
    y_train, y_valid = y.iloc[list(train_indices)], y.iloc[list(valid_indices)]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    score = log_loss(y_valid, y_valid_pred)
    print(f'fold {fold} log loss: {score}')
    valid_scores.append(score)

    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

In [ ]:
#結果を格納
y_pred = model.predict(test_features2,num_iteration=model.best_iteration) 
#最大値を選択
y_pred_max = np.argmax(y_pred, axis=1)

#predに予測結果を入れる
sub["AdoptionSpeed"] = y_pred_max
sub.to_csv('submission.csv', index=False)

In [ ]:
sub